## GCS first try
[website](https://cloud.google.com/storage/docs/reference/libraries)\
[documentation](https://cloud.google.com/python/docs/reference/storage/latest)

In [1]:
from google.oauth2 import service_account
from google.cloud import storage
from google.cloud.storage.bucket import Bucket
from io import BytesIO
from zipfile import ZipFile
import pandas as pd
import requests

# Set key_path to the path to the service account key file.
# This can be replaced by setting env export GOOGLE_APPLICATION_CREDENTIALS="KEY_PATH"
key_path = "./gcp_creds/gcp_credentials.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)


In [2]:
storage_client = storage.Client(credentials=credentials)

bucket_name = "tir101-group2"

bucket = storage_client.bucket(bucket_name)
if not bucket.exists():
    bucket.storage_class = "STANDARD"
    new_bucket = storage_client.create_bucket(bucket, location="asia-east1")

In [3]:
#upload from memory
def gcs_upload_df(bucket:Bucket,blob_name:str,df:pd.DataFrame) -> bool:
    """Uploads data from memory to the bucket."""
    blob = bucket.blob(blob_name)
    if blob.exists():
        print('file exist in gcp')
        return
    try:
        blob.upload_from_string(df.to_csv(),'text/csv')
        print('upload success')
    except Exception as e:
        print(e)

In [4]:
#upload from file

def gcs_file_upload(bucket:Bucket, source_name, dest_name):
    """Uploads a file to the bucket."""

    blob = bucket.blob(dest_name)
    if blob.exists():
        print('file exist in gcp')
        return
    try:
        blob.upload_from_filename(source_name)
        print('upload success')
    except Exception as e:
        print(e)

In [5]:
hist = pd.read_csv('YouBikeHis.csv')

In [6]:
test_url = hist['fileURL'][0]

In [7]:

resp = requests.get(test_url)

In [8]:
z = ZipFile(BytesIO(resp.content))

In [9]:
col_names = ['借用時段','借用站名','歸還時段','歸還站名','借用時間','紀錄日期']
df = pd.read_csv(z.open(z.filelist[0]),names=col_names)    

In [10]:
data_date = pd.to_datetime(df['紀錄日期'][0])
data_date

Timestamp('2023-11-22 00:00:00')

In [11]:
gcs_upload_df(bucket,f"ubike_history_{data_date.year}_{data_date.month}.csv",df)

upload success
